In [ ]:
#from IPython.core.display import display, HTML
from IPython.display import display, HTML
display(HTML("<style>.container { width: 98% !important; }</style>"))

In [ ]:
import numpy as np

### Input params

xdatcar_path = "./XDATCAR_40atoms_1100C"
element = 'O'

#xdatcar_path = "./XDATCAR_generated"
#element = 'Li'

dt = 1 # fs # timestep between coordinate outputs in MD

In [ ]:
def perform_PBC(L, dx_direct):
    L0 = np.sqrt(np.sum(np.power(L[0,:], 2)))
    L1 = np.sqrt(np.sum(np.power(L[1,:], 2)))
    L2 = np.sqrt(np.sum(np.power(L[2,:], 2)))
    L0hat = L[0,:]/L0
    L1hat = L[1,:]/L1
    L2hat = L[2,:]/L2
    #dx_cartesian  = dx_direct @ L # Conversion from direct to cartesian
    dx_cartesian  = dx_direct
    Natoms = dx_cartesian.shape[0]
    for i in range(Natoms):
        # correct along the 1st vector
        d0 = np.dot(dx_cartesian[i, :], L0hat)
        if (d0 >= L0/2):
            dx_cartesian[i,:] -= L[0,:]
        elif (d0 < -L0/2):
            dx_cartesian[i,:] += L[0,:]
        # correct along the 2nd vector
        d1 = np.dot(dx_cartesian[i,:],L1hat)
        if (d1 >= L1/2):
            dx_cartesian[i,:] -= L[1,:]
        elif (d1 < -L1/2):
            dx_cartesian[i,:] += L[1,:]
        # correct along the 3rd vector
        d2 = np.dot(dx_cartesian[i,:],L2hat)
        if (d2 >= L2/2):
            dx_cartesian[i,:] -= L[2,:]
        elif (d2 < -L2/2):
            dx_cartesian[i,:] += L[2,:]
        
    return dx_cartesian

In [ ]:
def get_x_info(xdatcar_path, desired_element):
    filename = xdatcar_path
    with open(filename, "r") as f:
        lines = f.readlines()
    
    L = np.zeros((3, 3))
    # Read the cell vectors
    counter = 0
    for i in range(2,5,1):
        templ = lines[i].split() 
        L[counter, :] = [float(templ[0]), float(templ[1]), float(templ[2])]
        counter += 1

    templ = lines[6].split() # for Nspecies
    Number_of_species = len(templ)
    Nspecies = np.zeros(Number_of_species, dtype=int)
    for i in range(Number_of_species):
        Nspecies[i] = int(templ[i])

    templ = lines[5].split() # for elements included 
    countelement = 0
    for i in range(Number_of_species):
        if templ[i] == desired_element:
            desired_element_sequence = countelement
            break
        countelement += 1
    
    Ndesiredelement = Nspecies[desired_element_sequence]
    
    #N1 = 7
    N2 = np.sum(Nspecies[0:desired_element_sequence]) + 1 # Rejecting reading other species and the "Direct configuration" line
    Nheaders = 7
    Natoms = np.sum(Nspecies)
    Nsnapshots = int((len(lines)-Nheaders)/(Natoms+1))
    x_total = np.zeros((Nsnapshots*Ndesiredelement, 3))
    counter = 0
    for i in range(Nsnapshots):
        N1 = Nheaders + i*(Natoms+1)
        for i in range(N1+N2, N1+N2+Ndesiredelement):
            templ = lines[i].split()
            x_total[counter, :] = [float(templ[0]), float(templ[1]), float(templ[2])]
            counter += 1

    return L, x_total, Ndesiredelement

In [ ]:
L, x_total, NLi = get_x_info(xdatcar_path, element)

In [ ]:
# Wish downselecting (?)
# x will be the array you need to use for further calculations
Neverythis = 1
Nsnapshots = int(x_total.shape[0]/NLi)
downselected_snapshots = np.arange(0, Nsnapshots, Neverythis)
Nsnaps = len(downselected_snapshots)
print("Number of snapshots that will be used is: %d" %Nsnaps)
x = np.zeros((len(downselected_snapshots)*NLi, 3))
counter = 0
for i in downselected_snapshots:
    idx1total = i*NLi
    idx2total = (i+1)*NLi
    idx1 = counter*NLi
    idx2 = (counter+1)*NLi
    x[idx1:idx2, :] = x_total[idx1total:idx2total, :]
    counter += 1

In [ ]:
# create time data of positions (nframes, natoms, 3)

r_cartesian = np.zeros((Nsnaps,NLi,3))
r_real = np.zeros((Nsnaps,NLi,3))

#types = np.zeros((nframes, natoms)) # atom types
#indx_frame = 0
for nt in range(Nsnaps):
    r_cartesian[nt] = x[0 + nt * NLi:NLi + nt * NLi]  @ L
    #types[indx_frame] = atoms.numbers
    #indx_frame += 1

dx = np.zeros((1, 3))

for m in range(Nsnaps):  # This block calculates r_real from r_cartesian (cancelling the PBC effects)
    if m == 0:  # positions at time step zero
        for n in range(NLi):
            r_real[m, n, 0] = r_cartesian[m, n, 0]
            r_real[m, n, 1] = r_cartesian[m, n, 1]
            r_real[m, n, 2] = r_cartesian[m, n, 2]
    else:
        for n in range(NLi):
            dx[0, 0] = r_cartesian[m, n, 0] - r_cartesian[m-1, n, 0]
            dx[0, 1] = r_cartesian[m, n, 1] - r_cartesian[m-1, n, 1]
            dx[0, 2] = r_cartesian[m, n, 2] - r_cartesian[m-1, n, 2]

            dx_PBC_corrected = perform_PBC(L, dx)

            r_real[m, n, 0] = r_real[m-1, n, 0] + dx_PBC_corrected[0, 0]
            r_real[m, n, 1] = r_real[m-1, n, 1] + dx_PBC_corrected[0, 1]
            r_real[m, n, 2] = r_real[m-1, n, 2] + dx_PBC_corrected[0, 2]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example data (replace with your actual data)
# r_real has shape (time, num_atoms, dimensions)
#time_steps = 100
#num_atoms = 5
#dimensions = 3
#r_real = np.random.random((time_steps, num_atoms, dimensions))  # Replace with actual data

# Generate time array
time = np.arange(r_real.shape[0])  # Assuming time index corresponds to the first axis

# Plot all atom-dimension combinations
plt.figure(figsize=(10, 8))
for atom in range(r_real.shape[1]):
    for dim in range(r_real.shape[2]):
        plt.plot(time, r_real[:, atom, dim], label=f'Atom {atom}, Dim {dim}')

# Customize plot
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Time Evolution of Atom-Dimension Combinations')
#plt.legend(loc='best', fontsize='small', ncol=2)
plt.grid()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ase.io import read, write

def crosscorFFT(x, y):
    assert len(x) == len(y), "Signals x and y must have the same length"
    N = len(x)
    F_x = np.fft.fft(x, n=2*N)  # 2*N because of zero-padding
    F_y = np.fft.fft(y, n=2*N)  # 2*N because of zero-padding
    PSD = F_x * F_y.conjugate()
    res = np.fft.ifft(PSD)
    res = (res[:N]).real   # now we have the cross-correlation in convention B
    n = N * np.ones(N) - np.arange(0, N) # divide res(m) by (N-m)
    return res / n # this is the cross-correlation in convention A

def msd_fft_cross(r1, r2, n):
    """
    Following section 4 (4.1, 4.2) in this paper:
    https://www.neutron-sciences.org/articles/sfn/abs/2011/01/sfn201112010/sfn201112010.html

    Args:
        r : Numpy array of positions (nframes, natoms, 3)
        n : Index of particular atom to calculate signal for
    """
    #N=len(r)
    
    # Check if dimensions of r1 and r2 are equal
    if r1.shape != r2.shape:
        raise ValueError("The dimensions of r1 and r2 must be the same")
    
    N = np.shape(r1)[0] # number of frames
    #D=np.square(r1).sum(axis=2) # sum over Cartesian coordinates
                               # now size (nframes, natoms)
    D = np.sum(r1 * r2, axis=2)
    
    #D = np.square(r)[:,0] # x-component

    #print(np.shape(D))

    # append a zero onto each atom signal
    #D=np.append(D,0) 
    nframes = np.shape(D)[0]
    natoms = np.shape(D)[1]
    npad = ((0, 1), (0, 0))
    D = np.pad(D, pad_width=npad, mode='constant', constant_values=0)
    # D is now size (nframes+1, natoms), since we appended a zero for each atom
    

    # calculate for each atom n
    #n = 0
    S2_xy = sum([crosscorFFT(r1[:, n, a], r2[:, n, a]) for a in range(r1.shape[2])]) # size (nframes,) for a particular atom
    S2_yx = sum([crosscorFFT(r2[:, n, a], r1[:, n, a]) for a in range(r1.shape[2])]) # size (nframes,) for a particular atom

    #print(np.shape(D))

    # sum over all times for all atoms
    #Q=2*D.sum() 
    Q=2*D.sum(axis=0) # now size (natoms,)

    Ndim = 3
    
    S1=np.zeros(N)
    for m in range(N):
        Q[n]=Q[n]-D[m-1,n]-D[N-m,n]
        S1[m]=Q[n]/(N-m)
    return (S1 - S2_xy - S2_yx) / (2 * Ndim)
    #return (S1 - 2*S2) / (2 * Ndim)

In [ ]:
# Let's get the MSD for each Li

msd_atom_ind = [msd_fft_cross(r_real[:-1, :, :], r_real[:-1, :, :], n) for n in range(0, NLi)]
msd_atom_ind = np.array(msd_atom_ind)

print(msd_atom_ind.shape)

In [ ]:
msd_atoms = np.mean(msd_atom_ind, axis=0)
print(msd_atoms.shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

x_values = np.arange(len(msd_atoms)) * dt  # x-values in fs

# Define fitting range
fit_start = int(0.10 * len(msd_atoms))  # 10% of the data
fit_end = int(0.54 * len(msd_atoms))   # 54% of the data

# Extract data for fitting
x_fit = x_values[fit_start:fit_end]
y_fit = msd_atoms[fit_start:fit_end]

# Perform linear fit
slope, intercept, r_value, p_value, std_err = linregress(x_fit, y_fit)

# Convert slope from A²/fs to cm²/s
slope_converted = slope / 10  # Conversion factor

# Plot the data and the fit
plt.figure(figsize=(8, 6))
plt.plot(x_values, msd_atoms, label='MSD data', color='blue')
plt.plot(x_fit, slope * x_fit + intercept, label=f'Fit line (slope={slope:.3e})', color='red')
plt.xlabel('Time (fs)')
plt.ylabel('MSD (Å²)')
plt.title('MSD vs. Time with Linear Fit')
plt.legend()
plt.grid()
plt.show()

# Report slope
print(f"Slope of the fitted line: {slope:.3e} Å²/fs")
print(f"Converted slope (Diffusion coefficient): {slope_converted:.3e} cm²/s")


In [ ]:
# Save data to an npz file
output_filename = "msd_data.npz"
np.savez(output_filename, MSD_values=msd_atoms, tau=x_values, D=slope_converted)

print(f"Data saved to {output_filename}")
